In [13]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="nous-hermes2:10.7b")


In [1]:
from langchain.chains import RetrievalQA

In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [15]:
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [6]:
from langchain_community.vectorstores import Chroma
persist_directory = 'aa/'

In [7]:
vectorstore = Chroma(persist_directory=persist_directory,embedding_function=embedding_model)

In [8]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [16]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [28]:
question = "what is oddo bhf"
result = qa_chain({"query": question})


In [29]:
result["result"]

"Oddo BHF is a European investment bank and asset management firm that offers a wide range of financial services to individual and institutional clients. Founded in 1961, the company has its headquarters in Paris, France, and operates across various countries including Germany, Luxembourg, Italy, and Spain.\n\nThe company provides services such as equity research, corporate finance, asset management, private banking, and trading services. Oddo BHF's research division offers market analysis and investment recommendations on various sectors and industries, helping clients make informed decisions about their investments.\n\nOddo BHF is part of the broader Oddo Group, which also includes other financial services providers like Oddo Asset Management and Oddo Private Equity. The company aims to provide tailored solutions for its clients' investment needs by leveraging its expertise in various fields and maintaining strong relationships with its clients."

In [30]:
result

{'query': 'what is oddo bhf',
 'result': "Oddo BHF is a European investment bank and asset management firm that offers a wide range of financial services to individual and institutional clients. Founded in 1961, the company has its headquarters in Paris, France, and operates across various countries including Germany, Luxembourg, Italy, and Spain.\n\nThe company provides services such as equity research, corporate finance, asset management, private banking, and trading services. Oddo BHF's research division offers market analysis and investment recommendations on various sectors and industries, helping clients make informed decisions about their investments.\n\nOddo BHF is part of the broader Oddo Group, which also includes other financial services providers like Oddo Asset Management and Oddo Private Equity. The company aims to provide tailored solutions for its clients' investment needs by leveraging its expertise in various fields and maintaining strong relationships with its client

Test the LLM without using RAG to observe the difference

In [31]:

llm2 = Ollama(model="nous-hermes2:10.7b")

llm2.invoke("what is oddo bhf")

'I\'m afraid I don\'t have any information on "oddo bhf". It could be a typo or an obscure term that hasn\'t been introduced to me yet. If you provide more context, I would be happy to help you with your inquiry.'